# Authors

* Sergey Golubev
* Andrey Vagin

We have taken given [tutorial](https://www.assemblyai.com/blog/end-to-end-speech-recognition-pytorch/) as base for our task.

# Imports

In [44]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torch.utils.data as data
import torch.nn.functional as F
import torchaudio
from comet_ml import Experiment
import os
import json
from typing import List

# Dataset Downloading

We decided to take `SpeechCommands` [dataset](https://arxiv.org/abs/1804.03209). It consists of 35 classes which are spoken short english words. 

['backward', 'bed', 'bird', 'cat', 'dog', 'down', 'eight', 'five', 'follow', 'forward', 'four', 'go', 'happy', 'house', 'learn', 'left', 'marvin', 'nine', 'no', 'off', 'on', 'one', 'right', 'seven', 'sheila', 'six', 'stop', 'three', 'tree', 'two', 'up', 'visual', 'wow', 'yes', 'zero']

In [23]:
# At first we tested our model on binary classification case and
# then apllied the results to the whole dataset.
"""
'SpeechCommands'        - full dataset          (51088 train, 6798 valid, 6835 test)
'SpeechCommandsBinary'  - only 'yes' and 'go'   ( 3721 train,  521 valid,  507 test)
"""
DATASET_FOLDER='SpeechCommands'

In [21]:
def download_data(dataset_folder, download=False):
    if not os.path.isdir("./data"):
        os.makedirs("./data")

    train_dataset = torchaudio.datasets.SPEECHCOMMANDS(root='./data', folder_in_archive=dataset_folder, url='speech_commands_v0.01', download=download, subset='training')
    valid_dataset = torchaudio.datasets.SPEECHCOMMANDS(root='./data', folder_in_archive=dataset_folder, url='speech_commands_v0.01', download=download, subset='validation')
    test_dataset = torchaudio.datasets.SPEECHCOMMANDS(root='./data', folder_in_archive=dataset_folder, url='speech_commands_v0.01', download=download, subset='testing')
    return train_dataset, valid_dataset, test_dataset

In [24]:
train_dataset, valid_dataset, test_dataset = download_data(DATASET_FOLDER)
print(len(train_dataset), len(valid_dataset), len(test_dataset), )

51088 6798 6835


# Data Processing

Model will be trained on mel spectrograms obtained from .wav files of the dataset.

Spectrograms will be splitted into chunks and our model will make a prediction for each of such chunks. Each letter will last for several chunks and there is usual pause or separation between differnet letters, for this we need special `<BLANK>` token which will represent these pauses.

Folllowing methods make indet_to_char or vice versa transformations.

In [ ]:
char_map_str = """
<BLANK> 0
a 1
b 2
c 3
d 4
e 5
f 6
g 7
h 8
i 9
j 10
k 11
l 12
m 13
n 14
o 15
p 16
q 17
r 18
s 19
t 20
u 21
v 22
w 23
x 24
y 25
z 26
"""

# To be assigned in TextTranform constructor.
BLANK_LABEL = None

class TextTransform:
    """Maps characters to their indices, and vice versa."""
    def __init__(self):
        self.char_to_index = {}
        self.index_to_char = {}
        global BLANK_LABEL
        for line in char_map_str.strip().split('\n'):
            char, index = line.split()
            self.char_to_index[char] = int(index)
            self.index_to_char[int(index)] = char
            if not BLANK_LABEL and char == '<BLANK>':
                BLANK_LABEL = int(index)
        self.index_to_char[BLANK_LABEL] = ' '

    def text_to_int(self, text: List[str]):
        """Uses a character map to convert text to an integer sequence."""
        int_sequence = []
        for c in text:
            if c == ' ':
                ind = self.char_to_index['<BLANK>']
            else:
                ind = self.char_to_index[c]
            int_sequence.append(ind)
        return int_sequence

    def int_to_text(self, labels: List[int]):
        """Uses a character map and convert integer labels to an text sequence."""
        string = []
        for i in labels:
            string.append(self.index_to_char[i])
        return ''.join(string).replace('<BLANK>', ' ')


train_audio_transforms = torchaudio.transforms.MelSpectrogram(sample_rate=16000, n_mels=128)
valid_audio_transforms = torchaudio.transforms.MelSpectrogram()
text_transform = TextTransform()

In [6]:
# How these methods work:
word_start = "yes"
index = text_transform.text_to_int(word_start)
word_recovered = text_transform.int_to_text(index)

print(word_start, "-->", index, "-->", word_recovered)

yes --> [25, 5, 19] --> yes


The function __data_processing()__ will be called in Data Loaders' __collate_fn__.

Data is represented as tuple(wave, sample_rate, utterance (label), speaker id, utterance number)

In [8]:
sample = train_dataset.__getitem__(n=2)
sample

(tensor([[-0.0025, -0.0021, -0.0017,  ..., -0.0030, -0.0033, -0.0031]]),
 16000,
 'bed',
 '004ae714',
 1)

Here we will transform the data into convenient form for model training.

In [17]:
def data_processing(data, data_type="train"):
    spectrograms = []
    labels = []
    input_lengths = []
    label_lengths = []
    for (waveform, _, utterance, _, _) in data:
        if data_type == 'train':
            spec = train_audio_transforms(waveform).squeeze(0).transpose(0, 1)
        else:
            spec = valid_audio_transforms(waveform).squeeze(0).transpose(0, 1)
        spectrograms.append(spec)
        # labels are lists of integer character ids
        label = torch.Tensor(text_transform.text_to_int(utterance.lower()))
        labels.append(label)
        # input_lengths, label_lengths are used in loss function
        input_lengths.append(spec.shape[0] // 2)
        label_lengths.append(len(label))

    spectrograms = nn.utils.rnn.pad_sequence(spectrograms, batch_first=True).unsqueeze(1).transpose(2, 3)
    labels = nn.utils.rnn.pad_sequence(labels, batch_first=True)

    return spectrograms, labels, input_lengths, label_lengths

In [10]:
data_processing((sample,))

(tensor([[[[0.0000e+00, 0.0000e+00, 0.0000e+00,  ..., 0.0000e+00,
            0.0000e+00, 0.0000e+00],
           [1.3406e-02, 1.1594e-02, 1.6736e-02,  ..., 1.7414e-02,
            1.6212e-02, 2.0798e-02],
           [7.2180e-02, 6.2425e-02, 9.0109e-02,  ..., 9.3760e-02,
            8.7287e-02, 1.1198e-01],
           ...,
           [5.7900e-06, 2.8763e-06, 3.3100e-06,  ..., 2.9408e-06,
            2.1657e-06, 3.0014e-06],
           [9.8653e-06, 2.7193e-06, 1.2482e-06,  ..., 6.7003e-07,
            5.9030e-07, 1.4782e-06],
           [9.3147e-06, 2.3657e-07, 1.7895e-07,  ..., 2.5175e-07,
            8.9278e-08, 2.5031e-06]]]]),
 tensor([[2., 5., 4.]]),
 [37],
 [3])

## CER - Character Error Rate

These methods are needed to count metrics for model evaluation.

In [16]:
def _levenshtein_distance(ref, hyp):
    """Levenshtein distance is a string metric for measuring the difference
    between two sequences. Informally, the levenshtein disctance is defined as
    the minimum number of single-character edits (substitutions, insertions or
    deletions) required to change one word into the other. We can naturally
    extend the edits to word level when calculate levenshtein disctance for
    two sentences.
    """
    m = len(ref)
    n = len(hyp)

    # special case
    if ref == hyp:
        return 0
    if m == 0:
        return n
    if n == 0:
        return m

    if m < n:
        ref, hyp = hyp, ref
        m, n = n, m

    # use O(min(m, n)) space
    distance = np.zeros((2, n + 1), dtype=np.int32)

    # initialize distance matrix
    for j in range(0,n + 1):
        distance[0][j] = j

    # calculate levenshtein distance
    for i in range(1, m + 1):
        prev_row_idx = (i - 1) % 2
        cur_row_idx = i % 2
        distance[cur_row_idx][0] = i
        for j in range(1, n + 1):
            if ref[i - 1] == hyp[j - 1]:
                distance[cur_row_idx][j] = distance[prev_row_idx][j - 1]
            else:
                s_num = distance[prev_row_idx][j - 1] + 1
                i_num = distance[cur_row_idx][j - 1] + 1
                d_num = distance[prev_row_idx][j] + 1
                distance[cur_row_idx][j] = min(s_num, i_num, d_num)

    return distance[m % 2][n]


def char_errors(reference, hypothesis, ignore_case=False, remove_space=False):
    """Compute the levenshtein distance between reference sequence and
    hypothesis sequence in char-level.
    :param reference: The reference sentence.
    :type reference: basestring
    :param hypothesis: The hypothesis sentence.
    :type hypothesis: basestring
    :param ignore_case: Whether case-sensitive or not.
    :type ignore_case: bool
    :param remove_space: Whether remove internal space characters
    :type remove_space: bool
    :return: Levenshtein distance and length of reference sentence.
    :rtype: list
    """
    if ignore_case == True:
        reference = reference.lower()
        hypothesis = hypothesis.lower()

    join_char = ' '
    if remove_space == True:
        join_char = ''

    reference = join_char.join(filter(None, reference.split(' ')))
    hypothesis = join_char.join(filter(None, hypothesis.split(' ')))

    edit_distance = _levenshtein_distance(reference, hypothesis)
    return float(edit_distance), len(reference)


def cer(reference, hypothesis, ignore_case=False, remove_space=False):
    """Calculate charactor error rate (CER). CER compares reference text and
    hypothesis text in char-level. CER is defined as:
    .. math::
        CER = (Sc + Dc + Ic) / Nc
    where
    .. code-block:: text
        Sc is the number of characters substituted,
        Dc is the number of characters deleted,
        Ic is the number of characters inserted
        Nc is the number of characters in the reference
    We can use levenshtein distance to calculate CER. Chinese input should be
    encoded to unicode. Please draw an attention that the leading and tailing
    space characters will be truncated and multiple consecutive space
    characters in a sentence will be replaced by one space character.
    :param reference: The reference sentence.
    :type reference: basestring
    :param hypothesis: The hypothesis sentence.
    :type hypothesis: basestring
    :param ignore_case: Whether case-sensitive or not.
    :type ignore_case: bool
    :param remove_space: Whether remove internal space characters
    :type remove_space: bool
    :return: Character error rate.
    :rtype: float
    :raises ValueError: If the reference length is zero.
    """
    edit_distance, ref_len = char_errors(reference, hypothesis, ignore_case,
                                         remove_space)

    if ref_len == 0:
        raise ValueError("Length of reference should be greater than 0.")

    cer = float(edit_distance) / ref_len
    return cer

# Building a Model

## NN Architecture

Combined model architecture was used here.

CNNs are great at extracting abstract features, they are also good when applied to audio spectrograms as they are also represented as images. Residual connections are used here as they help make “flatter” loss surface of the model. Residual connections also helps the model learn faster and generalize better.

RNNs are good at sequence modeling what is needed working with audio. BiRNN's are used next to take to consideration context of previous and next cells.

We use Layer Normalization, not Batch Normalization, because BN is hard to use with sequence data, with small batch sizes, and it's hard to paralellize a NN with BN.

This is due to the dependency on batches. Layer Normalization removes this dependency. It computes the normalization based on the layers inside of the batches.

LN briefly: Input values in all neurons in the same layer are normalized for each data sample.
So, all values in neurons of the same layer will have the same mean and variance.

LN is can deal with sequence data, doesn't depend on batch size, and is easily paralellized.
However, LN sometimes performs worse than BN with CNNs.

In [7]:
class CNNLayerNorm(nn.Module):
    """Layer Normalization"""
    
    def __init__(self, n_features):
        super(CNNLayerNorm, self).__init__()
        self.layer_norm = nn.LayerNorm(normalized_shape=n_features)
        """About normalized_shape parameter of nn.LayerNorm:
        If a single integer is used, it is treated as a singleton list, and this module will normalize
        over the last dimension which is expected to be of that specific size.
        """

    def forward(self, x):
        # x (batch, channel, feature, time)
        x = x.transpose(2, 3).contiguous() # (batch, channel, time, feature)
        x = self.layer_norm(x)
        return x.transpose(2, 3).contiguous() # (batch, channel, feature, time) 

class ResidualCNN(nn.Module):
    """ Residual CNN inspired by https://arxiv.org/pdf/1603.05027.pdf
        except with layer norm instead of batch norm """
    
    def __init__(self, in_channels, out_channels, kernel, stride, dropout, n_features):
        super(ResidualCNN, self).__init__()

        self.layer_norm1 = CNNLayerNorm(n_features)
        self.dropout1 = nn.Dropout(dropout)
        self.cnn1 = nn.Conv2d(in_channels, out_channels, kernel, stride, padding=kernel//2)

        self.layer_norm2 = CNNLayerNorm(n_features)
        self.dropout2 = nn.Dropout(dropout)
        self.cnn2 = nn.Conv2d(out_channels, out_channels, kernel, stride, padding=kernel//2)

    def forward(self, x):
        residual = x  # (batch, channel, feature, time)
        out = self.layer_norm1(x)
        out = F.gelu(out)
        out = self.dropout1(out)
        out = self.cnn1(out)
        out = self.layer_norm2(out)
        out = F.gelu(out)
        out = self.dropout2(out)
        out = self.cnn2(out)
        out += residual
        return out # (batch, channel, feature, time)
        
class BidirectionalGRU(nn.Module):

    def __init__(self, rnn_dim, hidden_size, dropout, batch_first):
        super(BidirectionalGRU, self).__init__()

        self.layer_norm = nn.LayerNorm(rnn_dim)
        self.BiGRU = nn.GRU(
            input_size=rnn_dim, hidden_size=hidden_size,
            num_layers=1, batch_first=batch_first, bidirectional=True)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        out = self.layer_norm(x)
        out = F.gelu(out)
        out, _ = self.BiGRU(out)
        out = self.dropout(out)
        return out


class SpeechRecognitionModel(nn.Module):
    """Speech Recognition Model Inspired by DeepSpeech 2"""

    def __init__(self, n_cnn_layers, n_rnn_layers, rnn_dim, n_class, n_features, stride=2, dropout=0.1):
        super(SpeechRecognitionModel, self).__init__()
        n_features = n_features // 2

        # cnn for extracting heirachal features
        self.cnn = nn.Conv2d(1, 32, 3, stride=stride, padding=3 // 2)

        # n residual cnn layers with filter size of 32
        self.rescnn_layers = nn.Sequential(*[
            ResidualCNN(32, 32, kernel=3, stride=1, dropout=dropout, n_features=n_features) 
            for _ in range(n_cnn_layers)
        ])
        self.fully_connected = nn.Linear(n_features*32, rnn_dim)
        
        self.birnn_layers = nn.Sequential(*[
            BidirectionalGRU(rnn_dim=rnn_dim if i==0 else rnn_dim*2,
                             hidden_size=rnn_dim, dropout=dropout, batch_first=i==0)
            for i in range(n_rnn_layers)
        ])
        self.classifier = nn.Sequential(
            nn.Linear(rnn_dim*2, rnn_dim),  # birnn returns rnn_dim*2
            nn.GELU(),
            nn.Dropout(dropout),
            nn.Linear(rnn_dim, n_class)
        )

    def forward(self, x):
        x = self.cnn(x)
        x = self.rescnn_layers(x)
        sizes = x.size()
        x = x.view(sizes[0], sizes[1] * sizes[2], sizes[3])  # (batch, feature, time)
        x = x.transpose(1, 2) # (batch, time, feature)
        x = self.fully_connected(x)
        x = self.birnn_layers(x)
        x = self.classifier(x)
        return x

# Decoder

This decoder works in the following way: it takes predicted chunks of audio and glue same indexes.

`[ 8,  8,  0,  0,  1,  1,  1, 16, 16, 16, 16,  0,  0,  0,  0, 16, 16, 16, 25, 25,  0,  0,  0,  0,  0,  0,  0,  0] --> 'happy'`

0 index states for the `<BLANK>` symbol.

8 -> h, 1 -> a, 16 -> p, 25 -> y

In [8]:
def GreedyDecoder(output, labels, label_lengths, blank_label=BLANK_LABEL, collapse_repeated=True):
    arg_maxes = torch.argmax(output, dim=2)
    decodes = []
    targets = []
    for i, args in enumerate(arg_maxes):
        decode = []
        targets.append(text_transform.int_to_text(labels[i][:label_lengths[i]].tolist()))
        for j, index in enumerate(args):
            if index != blank_label:
                if collapse_repeated and j != 0 and index == args[j -1]:
                    continue
                decode.append(index.item())
        decodes.append(text_transform.int_to_text(decode))
    return decodes, targets

# Training

In [15]:
def get_device():
    use_cuda = torch.cuda.is_available()
    torch.manual_seed(7)
    device = torch.device("cuda" if use_cuda else "cpu")
    return device, use_cuda

In [ ]:
class IterMeter(object):
    """Keeps track of total iterations. Used for Comet.ml"""
    def __init__(self):
        self.val = 0

    def step(self):
        self.val += 1

    def get(self):
        return self.val


def train(model, device, train_loader, criterion, optimizer, scheduler, epoch, iter_meter, experiment):
    model.train()
    data_len = len(train_loader.dataset)
    with experiment.train():
        for batch_idx, _data in enumerate(train_loader):
            spectrograms, labels, input_lengths, label_lengths = _data 
            spectrograms, labels = spectrograms.to(device), labels.to(device)

            optimizer.zero_grad()

            output = model(spectrograms)  # (batch, time, n_class)
            output = F.log_softmax(output, dim=2)
            output = output.transpose(0, 1) # (time, batch, n_class)

            loss = criterion(output, labels, input_lengths, label_lengths)
            loss.backward()

            experiment.log_metric('loss', loss.item(), step=iter_meter.get())
            experiment.log_metric('learning_rate', scheduler.get_last_lr(), step=iter_meter.get())

            optimizer.step()
            scheduler.step()   # scheduler step for oncecycle scheduler
            iter_meter.step()

            if batch_idx % 100 == 0 or batch_idx == data_len:
                print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                    epoch, batch_idx * len(spectrograms), data_len,
                    100. * batch_idx / len(train_loader), loss.item()))


def test(model, device, test_loader, criterion, epoch, iter_meter, experiment):
    print('\nEvaluating…')
    model.eval()
    test_loss = 0
    test_cer = []
    with experiment.test():
        with torch.no_grad():
            for I, _data in enumerate(test_loader):
                spectrograms, labels, input_lengths, label_lengths = _data 
                spectrograms, labels = spectrograms.to(device), labels.to(device)

                output = model(spectrograms)  # (batch, time, n_class)
                output = F.log_softmax(output, dim=2)
                output = output.transpose(0, 1) # (time, batch, n_class)

                loss = criterion(output, labels, input_lengths, label_lengths)
                test_loss += loss.item() / len(test_loader)

                decoded_preds, decoded_targets = GreedyDecoder(output.transpose(0, 1), labels, label_lengths)
                for j in range(len(decoded_preds)):
                    test_cer.append(cer(decoded_targets[j], decoded_preds[j]))

    avg_cer = sum(test_cer)/len(test_cer)
    experiment.log_metric('test_loss', test_loss, step=iter_meter.get())
    experiment.log_metric('cer', avg_cer, step=iter_meter.get())

    print('Test set: Average loss: {:.4f}, Average CER: {:4f}\n'.format(test_loss, avg_cer))


def train_test(hparams, experiment=Experiment(api_key='dummy_key', disabled=True)):

    experiment.log_parameters(hparams)

    train_dataset, valid_dataset, test_dataset = download_data(DATASET_FOLDER)

    device, use_cuda = get_device()

    kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}
    train_loader = data.DataLoader(dataset=train_dataset,
                                batch_size=hparams['batch_size'],
                                shuffle=True,
                                collate_fn=lambda x: data_processing(x, 'train'),
                                **kwargs)
    test_loader = data.DataLoader(dataset=test_dataset,
                                batch_size=hparams['batch_size'],
                                shuffle=False,
                                collate_fn=lambda x: data_processing(x, 'test'),
                                **kwargs)

    model = SpeechRecognitionModel(
        hparams['n_cnn_layers'], hparams['n_rnn_layers'], hparams['rnn_dim'],
        hparams['n_class'], hparams['n_feats'], hparams['stride'], hparams['dropout']
        ).to(device)

    print(model)
    print('Num Model Parameters', sum([param.nelement() for param in model.parameters()]))

    optimizer = optim.AdamW(model.parameters(), hparams['learning_rate'])
    criterion = nn.CTCLoss(blank=BLANK_LABEL).to(device)
    scheduler = optim.lr_scheduler.OneCycleLR(optimizer, max_lr=hparams['learning_rate'], 
                                            steps_per_epoch=int(len(train_loader)),
                                            epochs=hparams['epochs'],
                                            anneal_strategy='linear')

    iter_meter = IterMeter()
    for epoch in range(1, hparams['epochs'] + 1):
        train(model, device, train_loader, criterion, optimizer, scheduler, epoch, iter_meter, experiment)
        test(model, device, test_loader, criterion, epoch, iter_meter, experiment)

    return model

Experiments are monitored using Comet.ml:

In [16]:
comet_api_key = "KddYvSKPDO9U8K1lZUIUCgHjT"
project_name = "speech-recognition"
experiment_name = "asr-optim:adamw-scheduler:oncecycle-data:full-epochs:30"

if comet_api_key:
    experiment = Experiment(api_key=comet_api_key, project_name=project_name, parse_args=False)
    experiment.set_name(experiment_name)
    experiment.display()
else:
    experiment = Experiment(api_key='dummy_key', disabled=True)

COMET WARNING: Comet has disabled auto-logging functionality as it has been imported after the following ML modules: torch. Metrics and hyperparameters can still be logged using Experiment.log_metrics() and Experiment.log_parameters()
COMET WARNING: As you are running in a Jupyter environment, you will need to call `experiment.end()` when finished to ensure all metrics and code are logged before exiting.
COMET INFO: Experiment is live on comet.com https://www.comet.com/pigeongcc/speech-recognition/16f265cd92ab4718a5181cc4b75c9d8c



The main challenge to solve was to find hyperparameters with which model will learn. We tried different configurations until model started converging.

Run the training loop:

In [17]:
hparams = {
    "n_cnn_layers": 3,
    "n_rnn_layers": 3,
    "n_feats": 128,
    "stride": 2,
    "rnn_dim": 512,
    "n_class": len(text_transform.char_to_index),
    "dropout": 0.05,
    "learning_rate": 5e-4,
    "batch_size": 32,
    "epochs": 20,
    "scheduler_step_size": 3,
    "scheduler_gamma": 0.33,
    
}

model = train_test(hparams, experiment)

SpeechRecognitionModel(
  (cnn): Conv2d(1, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
  (rescnn_layers): Sequential(
    (0): ResidualCNN(
      (layer_norm1): CNNLayerNorm(
        (layer_norm): LayerNorm((64,), eps=1e-05, elementwise_affine=True)
      )
      (dropout1): Dropout(p=0.05, inplace=False)
      (cnn1): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (layer_norm2): CNNLayerNorm(
        (layer_norm): LayerNorm((64,), eps=1e-05, elementwise_affine=True)
      )
      (dropout2): Dropout(p=0.05, inplace=False)
      (cnn2): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    )
    (1): ResidualCNN(
      (layer_norm1): CNNLayerNorm(
        (layer_norm): LayerNorm((64,), eps=1e-05, elementwise_affine=True)
      )
      (dropout1): Dropout(p=0.05, inplace=False)
      (cnn1): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (layer_norm2): CNNLayerNorm(
        (layer_norm): LayerNorm((64,), eps=1e

In [19]:
experiment.end()

# Serialization

In [12]:
if not os.path.isdir("./models"):
    os.makedirs("./models")

def get_model_paths(experiment_name):
    model_folder_path = f'./models/{experiment_name}'
    model_path = f'{model_folder_path}/{experiment_name}.pth'
    hparams_path = f'{model_folder_path}/{experiment_name}.json'
    return model_folder_path, model_path, hparams_path

## Saving the Model

In [13]:
def save_model(experiment_name):
	model_folder_path, model_path, hparams_path = get_model_paths(experiment_name)

	if not os.path.isdir(model_folder_path):
		if not os.path.isdir("./models"):
			os.makedirs("./models")
		os.makedirs(model_folder_path)
		
	torch.save(model.state_dict(), model_path)

	with open(hparams_path, 'w') as f:
		json.dump(hparams, f)


In [22]:
save_model(experiment_name)

## Loading the Model

In [10]:
def load_model(experiment_name):
    _, model_path, hparams_path = get_model_paths(experiment_name)
    
    device, _ = get_device()

    with open(hparams_path, 'r') as f:
        hparams = json.load(f)

    model = SpeechRecognitionModel(
        hparams['n_cnn_layers'], hparams['n_rnn_layers'], hparams['rnn_dim'],
        hparams['n_class'], hparams['n_feats'], hparams['stride'], hparams['dropout']
        ).to(device)

    model.load_state_dict(torch.load(model_path))

    return model, hparams

# Inference

In [ ]:
experiment_name = 'asr-optim:adamw-scheduler:oncecycle-data:full-epochs:30'
model, hparams = load_model(experiment_name)

In [33]:
sample_idx = 2000

device, _ = get_device()
spectrogram, label, input_length, label_length = \
    data_processing((test_dataset.__getitem__(n=sample_idx),))
spectrogram, label = spectrogram.to(device), label.to(device)

word_label = text_transform.int_to_text(label.int().tolist()[0])
word_label

'happy'

In [34]:
model_inference = model
model_inference.eval()

output = model_inference(spectrogram)
output = F.log_softmax(output, dim=2)

decoded_preds, decoded_targets = GreedyDecoder(output, label, label_length, collapse_repeated=True)

In [36]:
torch.argmax(output, dim=2)

tensor([[ 8,  8,  0,  0,  1,  1,  1, 16, 16, 16, 16,  0,  0,  0,  0, 16, 16, 16,
         25, 25,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
          0,  0,  0,  0,  0]], device='cuda:0')

In [52]:
print(f"Target: {decoded_targets}")
print(f"Prediction: {decoded_preds}")

Target: ['happy']
Prediction: ['happy']


## Testing accuracy

In [30]:
experiment_name = 'asr-optim:adamw-scheduler:oncecycle-data:full-epochs:30'
model, hparams = load_model(experiment_name)
model.eval()

device, _ = get_device()

In [48]:
correct = 0

for sample in test_dataset:
    spectrogram, label, input_length, label_length = data_processing((sample,))

    spectrogram, label = spectrogram.to(device), label.to(device)

    output = model_inference(spectrogram)
    output = F.log_softmax(output, dim=2)

    decoded_preds, decoded_targets = GreedyDecoder(output, label, label_length, collapse_repeated=True)
    correct += (decoded_preds[0] == decoded_targets[0])

print('accuracy:', round(correct / len(test_dataset), 3))

accuracy: 0.925
